In [12]:
import re
import string
import pandas as pd
from functools import reduce
from math import log

## Simple example of [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
1. Example of corpus
2. Preprocessing and Tokenizing
3. Calculating bag of words
4. TF
5. IDF
6. TF-IDF

首先，我们看一个简单语料库的例子。这个语料库只有三个句子，句子本身也比较相近,也有重复的单词。为了方便处理，把语料库按照换行符进行了切分，并且删除了第一和最后的空句子。
First, let’s look at an example of a simple corpus.There are only three sentences in the corpus, and the sentences are quite similar.To facilitate processing, the corpus has been split by line breaks and the first and last empty sentences have been deleted.

In [127]:
#1
corpus = """
Simple example with Cats and Mouse and dogs
Another simple example with dogs and cats
Another simple example with mouse and cheese
""".split("\n")[1:-1]
print(corpus)

['Simple example with Cats and Mouse and dogs', 'Another simple example with dogs and cats', 'Another simple example with mouse and cheese']


处理三个句子，所有单词切分，变成小写的形式，便于接下来对单词本身进行处理。
Split the three sentences, and all words are converted to lower case forms to facilitate the following processing.

In [128]:
#2
l_A = corpus[0].lower().split()
l_B = corpus[1].lower().split()
l_C = corpus[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

['simple', 'example', 'with', 'cats', 'and', 'mouse', 'and', 'dogs']
['another', 'simple', 'example', 'with', 'dogs', 'and', 'cats']
['another', 'simple', 'example', 'with', 'mouse', 'and', 'cheese']


从语料库中获得词典，集合set中会自动去除重复的单词。
Form a dictionary from the sentences, with set and union. 

In [129]:
#3
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

{'cheese', 'example', 'and', 'simple', 'mouse', 'cats', 'dogs', 'with', 'another'}


统计每个句子中出现的单词以及单词出现的次数
 count the occurrence of words in each sentence

In [130]:
word_dict_A = dict.fromkeys(word_set, 0)
print(word_dict_A)

word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

print(word_dict_C)

pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

{'cheese': 0, 'example': 0, 'and': 0, 'simple': 0, 'mouse': 0, 'cats': 0, 'dogs': 0, 'with': 0, 'another': 0}
{'cheese': 1, 'example': 1, 'and': 1, 'simple': 1, 'mouse': 1, 'cats': 0, 'dogs': 0, 'with': 1, 'another': 1}


,cheese,example,and,simple,mouse,cats,dogs,with,another
0,0,1,2,1,1,1,1,1,0
1,0,1,1,1,0,1,1,1,1
2,1,1,1,1,1,0,0,1,1


词频就是一个词（term）在一个文档中（document）出现的次数（frequency）除以文档中的总词数，记为 tf(t,d)。这是一种最简单的定义方式。
词频的优点是简单，但缺点也很显然：
词频中没有包含词的位置信息，所以从词频的角度来看，"Mary is quicker than John"和"John is quicker than Mary"两条文档是完全一致的，
但显然它们的含义是完全相反的。词频没有考虑不同词的重要性一般是不一样的，比如停用词的词频都很高，但它们并不重要。

Term frequency (TF) is the number of times a term (word) appears in a document compared to the total number of words in the document. denoted as tf(t,d). This is the simplest possible definition.
The advantage of term frequency is its simplicity, but its disadvantages are also apparent: term frequency does not include information 
about the position of words, so from the perspective of term frequency, the documents "Mary is quicker than John" and "John is quicker than 
Mary" are completely identical, but it's obvious that their meanings are completely opposite. Term frequency does not consider that different
words are usually of different importance; for example, the frequency of stop words is generally high, but they are not important.

In [131]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

In [132]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

In [133]:
#Simple example with Cats and Mouse
tf_A

{'cheese': 0.0,
 'example': 0.125,
 'and': 0.25,
 'simple': 0.125,
 'mouse': 0.125,
 'cats': 0.125,
 'dogs': 0.125,
 'with': 0.125,
 'another': 0.0}

逆文档频率（Inverse Document Frequency, IDF）是一个术语在语料库中包含该术语的文档的比例。那些只存在于一小部分文档中的单词（例如，技术术语）比那些出现在所有文档中的单词（例如，a, the, and）获得更高的权重值。

Inverse Document Frequency: IDF of a term reflects the proportion of documents in the corpus that contain the term. Words unique to a small percentage of documents (e.g., technical jargon terms) receive higher importance values than words common across all documents (e.g., a, the, and).
## \#5 idf - inverse document frequency
idf is a measure of how much information the word provides
$$ \mathrm{idf}(t, D) =  \log \frac{N}{|\{d \in D: t \in d\}|} $$
- $N$: total number of strings in the corpus ${\displaystyle N={|D|}}$
- ${\displaystyle |\{d\in D:t\in d\}|}$  : number of strings where the term ${\displaystyle t}$ appears (i.e., ${\displaystyle \mathrm {tf} (t,d)\neq 0})$. If the term is not in the corpus, this will lead to a division-by-zero. It is therefore common to adjust the denominator to ${\displaystyle 1+|\{d\in D:t\in d\}|}$.
    
对于固定的语料库，N是固定的，一个词的在各个文献中出现的次数越多，其idf(t,D)就越小。所以那些很稀少的词的idf 值会很高，而像停用词这种出现频率很高的词idf值很低。
In a fixed corpus, N is constant. The more frequently a term appears in various documents, the smaller its IDF(t,D) value will be. Therefore, terms that are very rare will have high IDF values, while terms like stop words, which appear frequently in all documents, will have low IDF values.

In [134]:
def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1

    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [135]:
idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

idf

{'cheese': 1.0986122886681098,
 'example': 0.0,
 'and': 0.0,
 'simple': 0.0,
 'mouse': 0.4054651081081644,
 'cats': 0.4054651081081644,
 'dogs': 0.4054651081081644,
 'with': 0.0,
 'another': 0.4054651081081644}

## \# 6 tf-idf
Then tf–idf is calculated as
$$ {\displaystyle \mathrm {tfidf} (t,d,D)=\mathrm {tf} (t,d)\cdot \mathrm {idf} (t,D)} $$

TF-IDF就是将TF和IDF结合起来，就是简单地相乘。从公式可以分析出来，一个词 t 在某个文档d 中的tf-idf值：

当该词在少数文档中出现很多次的时候，其值接近最大值；（tf和idf都很大）
当该词在文档中出现次数少或者在很多文档中都出现时，其值较小；（tf或idf比较小）
当该词几乎在所有文档中都出现时，其值接近最小值。（idf很小，接近0）

In [136]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [137]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

In [138]:
pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,cheese,example,and,simple,mouse,cats,dogs,with,another
0,0.000000,0.0,0.0,0.0,0.050683,0.050683,0.050683,0.0,0.000000
1,0.000000,0.0,0.0,0.0,0.000000,0.057924,0.057924,0.0,0.057924
2,0.156945,0.0,0.0,0.0,0.057924,0.000000,0.000000,0.0,0.057924


# For clustering we must use tf-idf weights
the example above is just an example, in practice it is better to apply [TfidfVectorizer from sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

## Full text for clusterring

This corpus contain some strings about Google, some strings about TF-IDF from Wikipedia, and some strings about Taylor Swift . Just for example

In [142]:
all_text = """
Google and Facebook are strangling the free press to death. Democracy is the loser,Google is a great company
Your 60-second guide to security stuff Google touted today at Next '18
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo’s Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is. Google have been helping 
Android is from google, it is better than IOS from Apple
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
tfidf is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval text mining, and user modeling. 
The tf-idf value increases proportionally to the number of times a word appears in the document and is offset by the frequency of the word in the corpus
Taylor Alison Swift is an American singer-songwriter. 
Taylor is known for narrative songs about her personal life
Her songs have received widespread media coverage.
Taylor's album won four Grammy Awards.
Taylor is the youngest Grammy Album of the Year winner.
""".split("\n")[1:-1]

In [107]:
all_text1 = """
Google is a great company, Android is great open source product
Your 60 second guide to security stuff Google touted today
A Guide to Using Android Without Selling Your Soul to Google
Review: Lenovo Google Smart Display is pretty and intelligent
Google Maps user spots mysterious object submerged off the coast of Greece - and no-one knows what it is. Google have been helping 
Android is from google, it is better than IOS from Apple
In information retrieval, tf–idf or TFIDF, short for term frequency–inverse document frequency
tfidf is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
It is often used as a weighting factor in searches of information retrieval text mining, and user modeling. 
The tf-idf value increases proportionally to the number of times a word appears in the document and is offset by the frequency of the word in the corpus
Taylor Alison Swift is an American singer-songwriter. 
Taylor is known for narrative songs about her personal life
Her songs have received widespread media coverage.
Taylor's album won four Grammy Awards.
Taylor is the youngest Grammy Album of the Year winner,and her songs in the album are good.
""".split("\n")[1:-1]

## Preprocessing and tokenizing
Firstly, we must bring every chars to lowercase and remove all punctuation, because it's not important for our task, but is very harmful for clustering algorithm. 
After that, we'll split strings to array of words.

line = re.sub(r"[{}]".format(string.punctuation), " ", line)使用Python的re（正则表达式）模块来去除字符串中的所有标点符号。
re是Python中用于处理正则表达式的标准库模块。
sub是re模块中的一个函数，用于执行所谓的“替换操作”。它接受一个正则表达式模式，一个替换字符串，以及要操作的原始字符串，并在原始字符串中查找与正则表达式模式匹配的所有子串，并将它们替换为提供的替换字符串。
r"[{}]".format(string.punctuation)"是一个正则表达式模式，用于匹配字符串中的所有标点符号。string.punctuation是Python标准库string模块中的一个预定义字符串，包含了常见的标点符号，如句号、逗号、问号等。正则表达式中的方括号[]用于定义一个字符集，{}是格式化字符串的占位符，用于在运行时插入string.punctuation。
" "：这是用来替换匹配到的标点符号的字符串。在这里，它代表一个空格字符。
因此这行代码的作用是：遍历原始字符串line中的每个字符，检查它是否是标点符号；如果是，就用一个空格替换它；这个过程对字符串中的所有匹配项重复执行，最终删除所有的标点符号，只留下空格分隔的单词。

In [143]:
def preprocessing(line):
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    return line

In [144]:
#check the number of different words 
words = []
for line in all_text:
    line = line.lower()
    line = re.sub(r"[{}]".format(string.punctuation), " ", line)
    words = set(words).union(set(line.split()))
    
    
print(words)
len(words)  


{'review', 'modeling', 'received', 'are', 'display', 'greece', 'swift', 'alison', 'four', 'grammy', 'off', 'retrieval', 'what', 'that', 'used', 'year', 'it', 'or', 'collection', 'to', 'winner', 'facebook', 'her', 'text', 'media', 'coverage', 's', 'user', 'than', 'numerical', 'document', 'statistic', 'won', 'selling', 'company', 'without', 'term', '60', 'touted', 'knows', 'in', 'factor', 'increases', 'security', 'an', 'life', 'tf–idf', 'word', 'about', 'using', 'have', 'free', 'mysterious', 'as', 'appears', 'awards', 'helping', 'tfidf', 'weighting', 'searches', 'from', 'is', 'second', 'american', 'reflect', 'frequency', 'today', 'tf', 'intelligent', 'short', 'object', 'and', 'at', 'apple', '18', 'maps', 'been', 'often', 'times', 'of', 'number', 'no', 'how', 'a', 'stuff', 'smart', 'google', 'information', 'spots', 'important', 'widespread', 'death', 'the', 'loser', 'idf', 'pretty', 'proportionally', 'taylor', 'ios', 'personal', 'songs', 'submerged', 'guide', 'intended', 'offset', 'strang

129

Now, let's calculate tf-idf for this corpus

In [145]:
tfidf_vectorizer = TfidfVectorizer(preprocessor=preprocessing)
tfidf = tfidf_vectorizer.fit_transform(all_text)
#print(tfidf)

And train simple kmeans model with k = 3

K-means是一种聚类算法，它旨在将数据集分成K个簇（cluster），其中K是用户指定的簇数。算法的目标是将每个簇内的数据点之间的距离最小化，同时将不同簇之间的数据点之间的距离最大化。

The k-means method is a clustering algorithm used in unsupervised machine learning to partition a dataset into k distinct clusters, where k is a user-specified number of clusters. The goal of the algorithm is to minimize the distance between data points within the same cluster and maximize the distance between data points in different clusters.

In [146]:
kmeans = KMeans(n_clusters=3).fit(tfidf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [120]:
#print(kmeans.cluster_centers_)
pd.DataFrame([kmeans.labels_==0,kmeans.labels_==1,kmeans.labels_==2])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,False,False,False,False,False,False,False,False,False,False,True,True,True,True,False
1,False,False,False,False,True,False,True,True,True,True,False,False,False,False,True
2,True,True,True,True,False,True,False,False,False,False,False,False,False,False,False


Predictions

In [148]:
lines_for_predicting = ["tfidf","tfidf is great", "Google  android system is widely used.","Taylor swift is great singer"]
kmeans.predict(tfidf_vectorizer.transform(lines_for_predicting))

array([2, 0, 0, 1])

In [119]:
tfidf_vectorizer.transform(lines_for_predicting)
print(tfidf_vectorizer.transform(lines_for_predicting))

  (0, 98)	1.0
  (1, 98)	0.889354675226266
  (1, 47)	0.4572179585855993
  (2, 106)	0.6817346932385471
  (2, 47)	0.3043331735210504
  (2, 29)	0.40439480336945227
  (2, 7)	0.5282839866372372
  (3, 94)	0.42807465401963357
  (3, 93)	0.6094031292467774
  (3, 83)	0.6094031292467774
  (3, 47)	0.2720436411946478
